# Weakly Supervised Learning: Stack Overflow Tag Labeler

This is the first project for the book Weakly Supervised Learning, about using natural language processing (NLP) and weakly supervised learning (WSL) to build better models with less data. Using the non-code text of Stack Overflow posts (question and answers) to tag them using a multi-class, multi-label classifier based on 1D Convlutional Neural Networks.

In [1]:
import gc
import json
import math
import os
import re
import warnings

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pyarrow
import tensorflow as tf
import tensorflow_hub as hub

# Disable all warnings
warnings.filterwarnings("ignore")

# Download stop words
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rjurney/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Always use a random seed for reproducible results

Using random seeds throughout makes our work reproducible - so we get the same tokenization consistently, for example.

In [2]:
np.random.seed(seed=1337)

## Test GPU Support for Tensorflow/Keras

The first thing to do is to verify that our JupyterLab Python environment on our Data Science Engine EC2 instance is properly configured to work with its onboard GPU. We use `tensorflow.test.is_gpu_available` and `tensorflow.compat.v2.config.experimental.list_physical_devices` to verify the GPUs are working with Tensorflow.

In [3]:
gpu_avail = tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)
print(f'1 or more GPUs is available: {gpu_avail}')

avail_gpus = tf.compat.v2.config.experimental.list_physical_devices('GPU')
print(f'GPUs on tap: {avail_gpus}')

1 or more GPUs is available: True
GPUs on tap: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Widen the Max Width of Pandas Columns

We want to be able to see the text of the questions in our tables.

In [4]:
pd.set_option('display.max_colwidth', 50)

## Configure the Application

Setup the batch size, document size, token count for the tokenizer/embedding, the embedding dimensions and the test/train split ratio.

In [5]:
BATCH_SIZE  = 64
MAX_LEN     = 200
TOKEN_COUNT = 10000
EMBED_SIZE  = 50
TEST_SPLIT  = 0.3

## Load a Stratified Sample of Answered Stack Overflow Questions with Tags

We load a sample pulled from all answered questions from Stack Overflow. This data was converted from XML to parquet format via [code/stackoverflow/xml_to_parquet.py](stackoverflow/sample_json.spark.py) and then a more balanced stratified sample was computed for tags with over 50,000, 20,000, 10,000, 5,000 and 2,000 instances that reduced the maximum imbalance via oversampling using [code/stackoverflow/get_questions.spark.py](stackoverflow/get_questions.spark.py).

These scripts were run using a Spark cluster via Amazon Elastic MapReduce using 13 r5.12xlarge machines for about 24 hours at a cost of about \\$300 per full run, and about \\$1,500 overall to create and debug. Big data is expensive. The latter parts of the run could be computed on my own 6 core/64GB machine with NVM drives for data/Spark caching.

With this dataset the challenge isn't the number of records per say but rather the imbalance of the dataset if we wish to expand the number of tags the model can predict beyond low 3 digits. This leads us to some of the other techniques we'll cover involving weakly supervised learning.

In [6]:
# Tag limit defines which dataset to load - those with tags having at least 50K, 20K, 10K, 5K or 2K instances
TAG_LIMIT = 2000

# Pre-computed sorted list of tag/index pairs
sorted_all_tags = json.load(open(f'data/stackoverflow/08-05-2019/sorted_all_tags.{TAG_LIMIT}.json'))
max_index = sorted_all_tags[-1][0] + 1

# Load the parquet file using pyarrow for this tag limit, using the sorted tag index to specify the columns
posts_df = pd.read_parquet(
    f'data/stackoverflow/08-05-2019/Questions.Stratified.Final.{TAG_LIMIT}.parquet',
    columns=['_Body'] + ['label_{}'.format(i) for i in range(0, max_index)],
    engine='pyarrow'
)
posts_df.head(2)

,_Body,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,...,label_776,label_777,label_778,label_779,label_780,label_781,label_782,label_783,label_784,label_785
0,"[How, animate, Flutter, layout, keyboard, appearance, I, building, Flutter, app, mainly, iOS, One, views, text, field, iOS, keyboard, appears, tap, The, problem, layout, change, smoothly, like, native, iOS, apps, Instead, instantly, jumps, final, available, screen, height, even, keyboard, opening, animation, finishes, I, tried, wrapping, SafeArea, element, help, My, layout, code, How, I, make,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[Creating, Carousel, using, FutureBuilder, I, storing, images, Firestore, Document, would, like, load, using, FutureBuilder, Here, I, done, far, Using, code, I, display, image, without, errors, However, I, know, loop, snapshot, data, display, list, images, Below, firestore, structure, __PAD__, __PAD__, __PAD__, __PAD__, __PAD__, __PAD__, __PAD__, __PAD__, __PAD__, __PAD__, __PAD__, __PAD__, __...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
print(
    '{:,} Stackoverflow questions with a tag having at least 2,000 occurrences'.format(
        len(posts_df.index)
    )
)

1,554,788 Stackoverflow questions with a tag having at least 2,000 occurrences


## Sanity Check the Labels

There shouldn't be any rows that are all zeros - it throws off the metrics computed during training. There shouldn't be more than six tags for one question, there is a hard limit in the Stack Overflow UI.

In [8]:
test_matrix = posts_df[[f'label_{i}' for i in range(0, max_index)]].as_matrix()

tests = np.count_nonzero(test_matrix.sum(axis=1)), \
        test_matrix.sum(axis=1).shape[0], \
        test_matrix.sum(axis=1).min(), \
        test_matrix.sum(axis=1).max()

print(f'Non-zero rows: {tests[0]:,}, Total rows: {tests[1]:,}, Non-zero ratio: {tests[0]/tests[1]:,}, Least tags: {tests[2]:,}, Most tags: {tests[3]:,}')

Non-zero rows: 1,554,788, Total rows: 1,554,788, Non-zero ratio: 1.0, Least tags: 1, Most tags: 6


## Load the Precomputed Indexes for Converting Between Tag Indexes and Tags

At the end, when we visualize the predictions of our model versus the test data, we will use these indexes to convert from numeric predictions corresponding to tag indexes in a matrix to actual text tags. 

In [9]:
tag_index = json.load(open(f'data/stackoverflow/08-05-2019/tag_index.{TAG_LIMIT}.json'))
index_tag = json.load(open(f'data/stackoverflow/08-05-2019/index_tag.{TAG_LIMIT}.json'))

# Sanity check the different files
assert( len(tag_index.keys()) == len(index_tag.keys()) == len(sorted_all_tags) )

### Preparing the Data

We need to join the previously tokenized text back into a string for use in a Tokenizer, which provides useful properties. In addition, making the number of documents a multiple of batch size is a requirement for Tensorflow/Keras to split work among multiple GPUs and to use certain models such as Elmo.

In [10]:
# Convert label columns to numpy array
labels = posts_df[list(posts_df.columns)[1:]].to_numpy()

# Training_count must be a multiple of the BATCH_SIZE times the MAX_LEN for the Elmo embedding layer
highest_factor = math.floor(len(posts_df.index) / (BATCH_SIZE * MAX_LEN))
training_count = highest_factor * BATCH_SIZE * MAX_LEN
print('Highest Factor: {:,} Training Count: {:,}'.format(highest_factor, training_count))

documents = []
for body in posts_df[0:training_count]['_Body'].values.tolist():
    words = body.tolist()
    documents.append(' '.join(words))

labels = labels[0:training_count]

# Conserve RAM
del posts_df
gc.collect()

# Lengths for x and y match
assert( len(documents) == training_count == labels.shape[0] )

Highest Factor: 121 Training Count: 1,548,800


### Pad the Sequences

The data has already been truncated to 200 words per post but the tokenization using the top 10K words reduces this to below 200 in some documents. If any documents vary from 200 words, the data won't convert properly into a numpy matrix below.

Note that the string `__PAD__` has been used previously to pad the documents, so we use it here for words missing in the top `TOKEN_COUNT` words, in our `Tokenizer`.

The data has already been truncated to 200 words per post but the tokenization using the top 10K words reduces this to below 200 in some documents. If any documents vary from 200 words, the data won't convert properly into a *numpy* matrix below. 

In addition to converting the text to numeric sequences with a key, Keras’ *Tokenizer* class is handy for producing the final results of the model via the [*keras.preprocessing.text.Tokenizer.sequences_to_texts*](https://keras.io/preprocessing/text/#tokenizer) method. Then we use Keras’ [*keras.preprocessing.sequence.pad_sequences*](https://keras.io/preprocessing/sequence/#pad_sequences) method and check the output to ensure the sequences are all 200 items long or they won’t convert properly into a matrix. The string `__PAD__` has been used previously to pad the documents, so we reuse it here.

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(
    num_words=TOKEN_COUNT,
    oov_token='__PAD__'
)
tokenizer.fit_on_texts(documents)

sequences = tokenizer.texts_to_sequences(documents)

padded_sequences = pad_sequences(
    sequences,
    maxlen=MAX_LEN,
    dtype='int32',
    padding='post',
    truncating='post',
    value=1
)

# Conserve RAM
del documents
del sequences
gc.collect()

# Verify that all padded documents are now the same length
assert( min([len(x) for x in padded_sequences]) == MAX_LEN == max([len(x) for x in padded_sequences]) )

padded_sequences.shape

(1548800, 200)

## Load GloVe Embeddings

Stanford defines [GloVe Embeddings](https://nlp.stanford.edu/projects/glove/) as:

> GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space.

We'll try them out to see if they can beat our own embedding, specific to our data.

In [12]:
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

embeddings_index = dict(get_coefs(*o.strip().split()) for o in open('data/GloVe/glove.6B.50d.txt'))

In [13]:
# Create embeddings matrix
all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()

# Create embedding matrix using our vocabulary
word_index = tokenizer.word_index
print('word_index', word_index)
nb_words = min(TOKEN_COUNT, len(word_index))

# Initialize embedding matrix
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, 50))

# Loop through each word and get its embedding vector
missing_count = 0
too_short_count = 0
for word, i in word_index.items():
    if i >= TOKEN_COUNT: 
        too_short_count += 1
        continue # Skip words appearing less than the minimum allowed
    
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector
    else:
        missing_count += 1

print(missing_count, too_short_count, embedding_matrix[0])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Split Data into Test/Train Datasets

We need one dataset to train with and one separate dataset to test and validate our model with.  The oft used [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) makes it so.

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences,
    labels,
    test_size=TEST_SPLIT,
    random_state=1337
)

# Conserve RAM
del padded_sequences
del labels
gc.collect()

assert(X_train.shape[0] == y_train.shape[0])
assert(X_train.shape[1] == MAX_LEN)
assert(X_test.shape[0] == y_test.shape[0]) 
assert(X_test.shape[1] == MAX_LEN)

## Use Class Weights to Balance Uneven Label Counts

Although there has already been filtering and up-sampling of the data to restrict it to a sample of questions with at least one tag that occurs more than 2,000 times, there are still as much as a 8000:1 ratio between common and uncommon labels. Without class weights, the most common label will be much more likely to be predicted than the least common. Class weights will make the loss function consider uncommon classes more than frequent ones.

In [15]:
train_weight_vec = list(np.max(np.sum(y_train, axis=0))/np.sum(y_train, axis=0))
train_class_weights = {i: train_weight_vec[i] for i in range(y_train.shape[1])}

test_weight_vec = list(np.max(np.sum(y_test, axis=0))/np.sum(y_test, axis=0))
test_class_weights = {i: test_weight_vec[i] for i in range(y_test.shape[1])}

sorted(list(train_class_weights.items()), key=lambda x: x[1]), sorted(list(test_class_weights.items()), key=lambda x: x[1])

([(356, 1.0),
  (90, 1.09143374198603),
  (361, 1.3325384636111335),
  (546, 1.3808898878914375),
  (17, 1.6404179451032423),
  (516, 1.7734526285591337),
  (92, 1.81591025923478),
  (343, 2.411238420282789),
  (371, 3.1597290994473375),
  (1, 3.705567211149408),
  (309, 3.935423546731389),
  (89, 4.0107457118527226),
  (150, 4.613292910447761),
  (628, 4.779597951096936),
  (483, 5.11316170388751),
  (350, 5.191255970188422),
  (53, 5.266719914802982),
  (581, 5.4809376039011415),
  (459, 5.682465816385155),
  (471, 6.025158382066277),
  (552, 6.3403205128205125),
  (405, 6.612448188260463),
  (629, 7.392854473428508),
  (183, 7.418360459011475),
  (580, 7.432296363089871),
  (653, 7.646617703904136),
  (51, 7.8994489258046485),
  (34, 8.29425576519916),
  (753, 8.520761543762923),
  (56, 8.58659605868565),
  (766, 8.66406797477225),
  (378, 8.737544169611308),
  (622, 9.112677353970886),
  (457, 9.586063190540802),
  (590, 9.830931318954379),
  (647, 10.178964701039416),
  (773, 10.3

## Create a Performance Log for the Model

We will log the original performance as a reference point as well as the performance of the latest model to the current run.

In [16]:
try:
    simple_log
except NameError:
    simple_log = []

with open('data/simple_log.jsonl') as f:
    for line in f:
        simple_log.append(json.loads(line))

SEQUENCE = simple_log[-1]['sequence'] if len(simple_log) > 0 else 0

## Try a Simple CNN Model to Classify Questions to their Corresponding Tags

Now we’re ready to train a model to classify/label questions with tag categories. We start with a simple model with one `Conv1D`/`GlobalMaxPool1D`. We use the functional API and we’ve heavily parametrized the code so as to facilitate experimentation.

In [ ]:
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import binary_crossentropy, kld
from tensorflow.keras.optimizers import Adam

FILTER_COUNT        = 128
FILTER_SIZE         = 3
EPOCHS              = 8
ACTIVATION          = 'selu'
CONV_PADDING        = 'same'
STRIDES             = 1
EMBED_SIZE          = 50
EMBED_DROPOUT_RATIO = 0.1
CONV_DROPOUT_RATIO  = 0.1

EXPERIMENT_NAME = 'simple_cnn'

if EXPERIMENT_NAME == simple_log[-1]['name']:
    print('RENAME YOUR EXPERIMENT')
    raise Exception('RENAME YOUR EXPERIMENT')

SEQUENCE += 1

# Weights and Biases Monitoring
import wandb
from wandb.keras import WandbCallback
wandb.init(project="weakly-supervised-learning", name=EXPERIMENT_NAME)
config = wandb.config

config_dict = {
    'name': EXPERIMENT_NAME,
    'embedding': 'own',
    'architecture': 'Simple Conv1D',
    'epochs': EPOCHS,
    'batch_size': BATCH_SIZE,
    'filter_count': FILTER_COUNT,
    'filter_size': FILTER_SIZE,
    'activation': ACTIVATION,
    'conv_padding': CONV_PADDING,
    'sequence': SEQUENCE
}
print(config_dict)
config.update(
    config_dict
)

model = Sequential()

model.add(
    Embedding(
        TOKEN_COUNT,
        EMBED_SIZE, 
        input_length=X_train.shape[1],
        embeddings_initializer=RandomUniform(),
    )
)
# model.add(
#     Embedding(
#         TOKEN_COUNT,
#         EMBED_SIZE,
#         weights=[embedding_matrix],
#         input_length=MAX_LEN,
#         trainable=True
#     )
# )
model.add(Dropout(0.1))
model.add(
    Conv1D(
        FILTER_COUNT, 
        FILTER_SIZE, 
        padding=CONV_PADDING, 
        activation=ACTIVATION, 
        strides=1
    )
)
model.add(GlobalMaxPool1D())
model.add(
    Dense(
        y_train.shape[1],
        activation='sigmoid',
    )
)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'categorical_accuracy',
        tf.keras.metrics.CategoricalAccuracy(),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.AUC(),
        tf.keras.metrics.TruePositives(),
        tf.keras.metrics.FalsePositives(),
        tf.keras.metrics.TrueNegatives(),
        tf.keras.metrics.FalseNegatives(),
    ]
)
model.summary()

callbacks = [
    ReduceLROnPlateau(
        monitor='val_categorical_accuracy',
        factor=0.1,
        patience=1,
        verbose=1,
    ), 
    EarlyStopping(
        monitor='val_categorical_accuracy',
        patience=2,
        verbos=1,
    ), 
    ModelCheckpoint(
        filepath='models/cnn_tagger.weights.hdf5',
        monitor='val_categorical_accuracy',
        save_best_only=True,
        verbose=1,
    ),
    WandbCallback()
]

history = model.fit(X_train, y_train,
                    class_weight=train_class_weights,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_split=TEST_SPLIT,
                    callbacks=callbacks)

E0920 11:17:28.471515 140318888896320 jupyter.py:104] Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable


{'name': 'simple_cnn', 'embedding': 'own', 'architecture': 'Simple Conv1D', 'epochs': 8, 'batch_size': 64, 'filter_count': 128, 'filter_size': 3, 'activation': 'selu', 'conv_padding': 'same', 'sequence': 50}


wandb: Wandb version 0.8.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
W0920 11:17:36.214836 140318888896320 deprecation.py:323] From /home/rjurney/anaconda3/envs/deep/lib/python3.6/site-packages/tensorflow_core/python/keras/backend.py:4266: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 50)           500000    
_________________________________________________________________
dropout (Dropout)            (None, 200, 50)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 200, 128)          19328     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 786)               101394    
Total params: 620,722
Trainable params: 620,722
Non-trainable params: 0
_________________________________________________________________
Train on 758912 samples, validate on 325248 samples
Epoch 1/8
758912/758912 [==============================] - 150

## Create a Log of the Model's Performance

Metrics include names like *precision_66* which aren't consistent between runs. We fix these to cleanup our report on training the model. We also add an f1 score, then make a DataFrame to display the log. This could be extended in repeat experiments.

In [ ]:
def fix_metric_name(name):
    """Remove the trailing _NN, ex. precision_86"""
    if name[-1].isdigit():
        repeat_name = '_'.join(name.split('_')[:-1])
    else:
        repeat_name = name
    return repeat_name

def fix_value(val):
    """Convert from numpy to float"""
    return val.item() if isinstance(val, np.float32) else val

def fix_metric(name, val):
    repeat_name = fix_metric_name(name)
    py_val = fix_value(val)
    return repeat_name, py_val


log = {}
for name, val in zip(model.metrics_names, metrics):
    
    repeat_name, py_val = fix_metric(name, val)
    log[repeat_name] = py_val

# Add a name and sequence number and an F1 score
log.update({'name': EXPERIMENT_NAME})
log.update({'sequence': SEQUENCE})
log.update({'f1': (log['precision'] * log['recall']) / (log['precision'] + log['recall'])})

simple_log.append(log)

# Overwrite the old log
with open('data/simple_log.jsonl', 'w') as f:
    [f.write(json.dumps(l) + '\n') for l in simple_log]

pd.DataFrame([log])

## Plot the Epoch Accuracy

We want to know the performance at each epoch so that we don't train needlessly large numbers of epochs. 

In [ ]:
%matplotlib inline

new_history = {}
for key, metrics in history.history.items():
    new_history[fix_metric_name(key)] = metrics

import matplotlib.pyplot as plt

viz_keys = ['val_categorical_accuracy', 'val_precision', 'val_recall']
# summarize history for accuracy
for key in viz_keys:
    plt.plot(new_history[key])
plt.title('model accuracy')
plt.ylabel('metric')
plt.xlabel('epoch')
plt.legend(viz_keys, loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Train a Kim-CNN Model to Label Stack Overflow Questions

Once again we’re ready to train a model to classify/label questions with tag categories. The model is based on [Kim-CNN](https://arxiv.org/abs/1408.5882), a commonly used convolutional neural network for sentence and document classification. We use the functional API and we’ve heavily parametrized the code so as to facilitate experimentation. 

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras.layers import (
    Dense, Activation, Embedding, Flatten, MaxPool1D, GlobalMaxPool1D, Dropout, Conv1D, Input, concatenate
)
from tensorflow.keras.losses import binary_crossentropy, kld
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from keras_radam import RAdam

EXPERIMENT_NAME = 'kim_cnn_2000_3_4_5_7'

FILTER_COUNT        = 128
FILTER_SIZE         = [3, 4, 5, 7]
EPOCHS              = 5
ACTIVATION          = 'selu'
CONV_PADDING        = 'same'
EMBED_SIZE          = 50
EMBED_DROPOUT_RATIO = 0.1
CONV_DROPOUT_RATIO  = 0.1

if EXPERIMENT_NAME == simple_log[-1]['name']:
    print('RENAME YOUR EXPERIMENT')
    raise Exception('RENAME YOUR EXPERIMENT')

SEQUENCE += 1

# Weights and Biases Monitoring
import wandb
from wandb.keras import WandbCallback
wandb.init(project="weakly-supervised-learning", name=EXPERIMENT_NAME)
config = wandb.config

config.update(
    {
        'name': EXPERIMENT_NAME,
        'embedding': 'own',
        'architecture': 'Kim CNN',
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'filter_count': FILTER_COUNT,
        'filter_size': FILTER_SIZE,
        'activation': ACTIVATION,
        'conv_padding': CONV_PADDING,
        'sequence': SEQUENCE
    }
)

padded_input = Input(
    shape=(X_train.shape[1],),
    dtype='int32'
)

emb = Embedding(
    TOKEN_COUNT, 
    EMBED_SIZE,
    embeddings_initializer=RandomUniform(),
    input_length=X_train.shape[1]
)(padded_input)
# emb = Embedding(
#     TOKEN_COUNT,
#     EMBED_SIZE,
#     weights=[embedding_matrix],
#     input_length=MAX_LEN,
#     trainable=True,
# )(padded_input)
drp = Dropout(0.1)(emb)

# Create convlutions of different sizes
convs = []
for filter_size in FILTER_SIZE:
    f_conv = Conv1D(
        filters=FILTER_COUNT,
        kernel_size=filter_size,
        padding=CONV_PADDING,
        activation=ACTIVATION
    )(drp)
    f_pool = MaxPool1D(filter_size)(f_conv)
    convs.append(f_pool)

l_merge = concatenate(convs, axis=1)
l_conv = Conv1D(
    128,
    5,
    activation=ACTIVATION
)(l_merge)
l_pool = GlobalMaxPool1D()(l_conv)
l_flat = Flatten()(l_pool)
l_drp  = Dropout(CONV_DROPOUT_RATIO)(l_flat)
l_dense = Dense(
    128,
    activation=ACTIVATION
)(l_drp)
out_dense = Dense(
    y_train.shape[1],
    activation='sigmoid'
)(l_dense)

model = Model(inputs=padded_input, outputs=out_dense)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        tf.keras.metrics.CategoricalAccuracy(),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.AUC(),
        tf.keras.metrics.TruePositives(),
        tf.keras.metrics.FalsePositives(),
        tf.keras.metrics.TrueNegatives(),
        tf.keras.metrics.FalseNegatives(),
    ]
)
model.summary()

callbacks = [
    ReduceLROnPlateau(
        monitor='val_categorical_accuracy',
        factor=0.1,
        patience=1,
        verbose=1,
    ), 
    EarlyStopping(
        monitor='val_categorical_accuracy',
        patience=2,
        verbose=1,
    ), 
    ModelCheckpoint(
        filepath='models/cnn_tagger.weights.hdf5',
        monitor='val_categorical_accuracy',
        save_best_only=True,
        verbose=1,
    ),
    WandbCallback()
]

history = model.fit(X_train, y_train,
                    class_weight=train_class_weights,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_data=(X_test, y_test),
                    callbacks=callbacks)

In [ ]:
model = tf.keras.models.load_model('models/cnn_tagger.weights.hdf5')
metrics = model.evaluate(X_test, y_test)

In [ ]:
log = {}
for name, val in zip(model.metrics_names, metrics):
    
    repeat_name, py_val = fix_metric(name, val)
    log[repeat_name] = py_val

# Add a name and sequence number and an F1 score
log.update({'name': EXPERIMENT_NAME})
log.update({'sequence': SEQUENCE})
log.update({'f1': (log['precision'] * log['recall']) / (log['precision'] + log['recall'])})

simple_log.append(log)

# Overwrite the old log
with open('data/simple_log.jsonl', 'w') as f:
    [f.write(json.dumps(l) + '\n') for l in simple_log]

pd.DataFrame([log])

In [ ]:
%matplotlib inline

new_history = {}
for key, metrics in history.history.items():
    new_history[fix_metric_name(key)] = metrics

import matplotlib.pyplot as plt

viz_keys = ['val_categorical_accuracy', 'val_precision', 'val_recall']
# summarize history for accuracy
for key in viz_keys:
    plt.plot(new_history[key])
plt.title('model accuracy')
plt.ylabel('metric')
plt.xlabel('epoch')
plt.legend(viz_keys, loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Compare this Run to the 1st and Previous Run

To get an idea of performance we need to see where we started and where we just came from.

In [ ]:
# Compare to original
if len(simple_log) > 1:
    d2 = simple_log[-1]
    d1 = simple_log[0]
else:
    d1 = simple_log[0]
    d2 = simple_log[0]
log_diff_1 = {key: d2.get(key, 0) - d1.get(key, 0) for key in d1.keys() if key not in ['name', 'sequence']}
log_diff_1['current'] = d2['name']
log_diff_1['previous'] = d1['name']

# Compare to last run
if len(simple_log) > 1:
    d1 = simple_log[-2]
    d2 = simple_log[-1]
else:
    d1 = simple_log[0]
    d2 = simple_log[0]
    
log_diff_2 = {key: d2.get(key, 0) - d1.get(key, 0) for key in d1.keys() if key not in ['name', 'sequence']}
log_diff_2['current'] = d2['name']
log_diff_2['previous'] = d1['name']

df = pd.DataFrame.from_dict([log_diff_1, log_diff_2])
cols = df.columns.tolist()
cols.remove('previous')
cols.remove('current')
show_cols = ['previous', 'current'] + cols
df[show_cols]

## View the Last 10 Experiments

It can be helpful to see trends of performance among experiments.

In [ ]:
log_df = pd.DataFrame(simple_log)
log_df['f1'] = (log_df['precision'] * log_df['recall']) / (log_df['precision'] + log_df['recall'])

log_df[[
    'sequence',
    'name',
    'loss',
    'categorical_accuracy',
    'precision',
    'recall',
    'f1',
    'auc',
    'true_positives',
    'false_positives',
    'true_negatives',
    'false_negatives',
    'hinge',
    'mean_absolute_error',
]][0:10 if len(log_df) > 9 else len(log_df)]

## Check the Actual Prediction Outputs

It is not enough to know theoretical performance. We need to see the actual output of the tagger at different confidence thresholds.

In [ ]:
TEST_COUNT = 1000

X_test_text = tokenizer.sequences_to_texts(X_test[:TEST_COUNT])

y_test_tags = []
for row in y_test[:TEST_COUNT].tolist():
    tags = [index_tag[str(i)] for i, col in enumerate(row) if col == 1]
    y_test_tags.append(tags)

## Adjust the threshold for classification

This lets us see how well the model generalizes to labeling more classes.

In [ ]:
CLASSIFY_THRESHOLD = 0.5

y_pred = model.predict(X_test)
y_pred = (y_pred > CLASSIFY_THRESHOLD) * 1

y_pred_tags = []
for row in y_pred.tolist():
    tags = [index_tag[str(i)] for i, col in enumerate(row) if col > CLASSIFY_THRESHOLD]
    y_pred_tags.append(tags)

## See How Far off we are per Class

In [ ]:
np.around(y_pred, 0).sum(axis=0) - y_test.sum(axis=0)

### View Prediction Results

It is better to view the results in a DataFrame.

In [ ]:
prediction_tests = []
for x, y, z in zip(X_test_text, y_pred_tags, y_test_tags):
    prediction_tests.append({
        'Question': x,
        'Actual': ' '.join(sorted(z)),
        'Predictions': ' '.join(sorted(y)),
    })

pd.set_option('display.max_colwidth', 300)
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(prediction_tests)

## The Big Finish

That is the big finish!